<a href="https://colab.research.google.com/github/changyong93/Natural-language-processing-with-chat-bot/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%85%EB%AC%B8(12_1~3_NLP%EB%A5%BC_%EC%9C%84%ED%95%9C_%EC%8B%A0%EA%B2%BD%EB%A7%9D(CNN)_1D_CNN%EC%9C%BC%EB%A1%9C_IMDB_%EB%A6%AC%EB%B7%B0_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1D CNN으로 IMDB 리뷰 분류하기

In [ ]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words = vocab_size)

In [ ]:
print(X_train[:5])

In [ ]:
max_len = 200
X_train = pad_sequences(sequences=X_train, maxlen = max_len)
X_test = pad_sequences(sequences=X_test, maxlen = max_len)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train[:5]

In [ ]:
# 1D CNN 모델 서계

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Conv1D, GlobalMaxPool1D, Embedding

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = 256))
model.add(Dropout(rate = 0.3)) #전체 입력 중 학습마다 랜덤하게 30%의 샘플을 선택하여 제외 #과적합 방지
model.add(Conv1D(filters = 256, kernel_size = 3, strides = 1, padding = 'valid',activation = 'relu')) #padding(valid) = 패딩 안함
model.add(GlobalMaxPool1D()) # 합성곱odel = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = 256))
model.add(Dropout(rate = 0.3)) #전체 입력 중 학습마다 랜덤하게 30%의 샘플을 선택하여 제외 #과적합 방지
model.add(Conv1D(filters = 256, kernel_size = 3, strides = 1, padding = 'valid',activation = 'relu')) #padding(valid) = 패딩 안함
model.add(GlobalMaxPool1D()) # 합성곱
model.add(Dense(units = 128, activation = "relu"))
model.add(Dropout(rate = 0.5))
model.add(Dense(units = 1, activation = "sigmoid"))

In [ ]:
es = EarlyStopping(monitor = 'val_loss', patience = 3, verbose = 1, mode = 'min')
mc = ModelCheckpoint(filepath = "base_model.h5", monitor='val_acc', mode = "max", save_best_only=True)

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(x = X_train, y = y_train, epochs = 20, validation_data = (X_test,y_test), callbacks = [es,mc])

In [ ]:
loaded_model = load_model("base_model.h5")
print(f"테스트 정확도: {loaded_model.evaluate(x = X_test, y = y_test)[1]:.3f}%")